In [1]:
import more_constraints_sdr as api
import more_constraints_sdr_expl_pump as api2
import numpy as np
import scipy
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import cvxpy as cp

np.random.seed(0)

In [2]:
# Initialize parameters and SDR constraint
N_omega = 5
omega = np.linspace(-2, 2, N_omega)
N_z = 30
z = np.linspace(0, 3.5*10**-2, N_z)
green_fs = api.get_green_f(omega,z)
projection = np.zeros((N_omega, N_omega))
projections = []
sdr_def_constr = []
sdr_cst = []
for i in range(N_omega):
    for j in range(N_omega):
        proj_copy = projection.copy()
        proj_copy[i, j] = 1
        projections.append(sparse.csc_matrix(proj_copy))
        sdr_def_constr.append(api.sdr_def_constr(N_omega, N_z, sparse.csc_matrix(proj_copy)))
        if i == j:
            sdr_cst.append(2.)
        else:
            sdr_cst.append(0.)

In [3]:
# Propagators for testing out legitimacy of matrices
rand_pump = np.random.random(2*N_omega - 1)
beta = scipy.linalg.hankel(rand_pump[:N_omega], rand_pump[N_omega - 1:])
delta_k = np.diag(1.j*omega)
W_plus = [scipy.linalg.expm((delta_k + beta)*z[i]) for i in range(N_z)]
W_minus = [scipy.linalg.expm((delta_k - beta)*z[i]) for i in range(N_z)]
J = 0.25*(W_plus[-1]@W_plus[-1].conj().T + W_minus[-1]@W_minus[-1].conj().T - 2*np.eye(N_omega))
n = np.trace(J)
stacked_W_plus = np.vstack(W_plus)
stacked_W_minus = np.vstack(W_minus)
X = sparse.csc_matrix(np.vstack([J, stacked_W_plus, stacked_W_minus]))
Y = sparse.vstack([X, sparse.eye(N_omega)])
print("value of the objective function: ", n**2 - np.trace(J.conj().T@J))
print("photon number pair: ", n)

value of the objective function:  (1.2366843356267752e-05+0j)
photon number pair:  (0.01138059690855453+0j)


In [4]:
# Commutation of propagators
W_plus_no_comm = scipy.linalg.expm(delta_k*z[-1])@scipy.linalg.expm(beta*z[-1])
W_minus_no_comm = scipy.linalg.expm(delta_k*z[-1])@scipy.linalg.expm(-beta*z[-1])
J_no_comm = 0.25*(W_plus_no_comm@W_plus_no_comm.conj().T + W_minus_no_comm@W_minus_no_comm.conj().T - 2*np.eye(N_omega))
first_comm = -((z[-1]**2)/2)*(delta_k@beta - beta@delta_k)
second_comm = ((z[-1]**3)/6)*(2*(beta@first_comm - first_comm@beta) + (delta_k@first_comm - first_comm@delta_k))
test = W_plus_no_comm@scipy.linalg.expm(first_comm)@scipy.linalg.expm(second_comm)

In [5]:
# Plot
J_s = [0.25*(W_plus[i]@W_plus[i].conj().T + W_minus[i]@W_minus[i].conj().T - 2*np.eye(N_omega)) for i in range(N_z)]
traces = [np.trace(J_s[i]) for i in range(N_z)]

In [5]:
# Get the dynamics constraints, symplectic and definition of J constraints and constraints that fix the pump
real_plus_constr_mat = []
imag_plus_constr_mat = []
real_minus_constr_mat = []
imag_minus_constr_mat = []
sympl_real_constr = []
sympl_imag_constr = []
def_J_real_constr = []
def_J_imag_constr = []
real_fixed_pump_constr = []
imag_fixed_pump_constr = []
for i in range(len(projections)):    
    real_plus, imag_plus, real_minus, imag_minus = api.get_dynamics_sdr(omega, z, projections[i])
    real_plus_constr_mat += real_plus
    imag_plus_constr_mat += imag_plus
    real_minus_constr_mat += real_minus
    imag_minus_constr_mat += imag_minus
    sympl_real, sympl_imag = api.sympl_constr_sdr(N_omega, N_z, projections[i])
    sympl_real_constr += sympl_real
    sympl_imag_constr += sympl_imag
    real_def_J, imag_def_J = api.def_J_constr(N_omega, N_z, projections[i])
    def_J_real_constr.append(real_def_J)
    def_J_imag_constr.append(imag_def_J)
    real_fix_pump, imag_fix_pump = api.fix_pump_constr(N_omega, N_z, projections[i], beta)
    real_fixed_pump_constr.append(real_fix_pump)
    imag_fixed_pump_constr.append(imag_fix_pump)

In [6]:
# The rest of the constraints (objective function is positive, photon number constraint, constraint that prior photon number is always less than the end)
obj_f = api.obj_f_sdr(N_omega, N_z, n)
photon_nbr_constr = api.photon_nbr_sdr_constr(N_omega, N_z, n)
#inf_constr_photon_nbr = api.inf_trace_photon_nbr(N_omega, N_z, n)

In [7]:
[(Y.conj().T@real_plus_constr_mat[i]@Y).trace() for i in range(len(real_plus_constr_mat)) if np.abs((Y.conj().T@real_plus_constr_mat[i]@Y).trace()) > 6*10**-6]

[]

In [8]:
# List containing all the constraints
constr_list = real_plus_constr_mat + imag_plus_constr_mat + real_minus_constr_mat + imag_minus_constr_mat + sympl_real_constr + sympl_imag_constr + def_J_real_constr + def_J_imag_constr + sdr_def_constr + real_fixed_pump_constr + imag_fixed_pump_constr
constr_list.append(photon_nbr_constr)

In [49]:
[((Y.conj().T@constr_list[i]@Y).trace(), i) for  i in range(len(constr_list)) if np.abs((Y.conj().T@constr_list[i]@Y).trace()) > 3*10**-5]

[((2+0j), 12050),
 ((2+0j), 12056),
 ((2+0j), 12062),
 ((2+0j), 12068),
 ((2+0j), 12074)]

In [9]:
# Make the cvxpy model
variable = cp.Variable(shape = ((2*N_z + 3)*N_omega, (2*N_z + 3)*N_omega), complex = True)
constraints = [variable >> 0]
constraints += [cp.real(cp.trace(real_plus_constr_mat[i]@variable)) == 0 for i in range(len(real_plus_constr_mat))]
constraints += [cp.real(cp.trace(imag_plus_constr_mat[i]@variable)) == 0 for i in range(len(imag_plus_constr_mat))]
constraints += [cp.real(cp.trace(real_minus_constr_mat[i]@variable)) == 0 for i in range(len(real_minus_constr_mat))]
constraints += [cp.real(cp.trace(imag_minus_constr_mat[i]@variable)) == 0 for i in range(len(imag_minus_constr_mat))]
constraints += [cp.real(cp.trace(sympl_real_constr[i]@variable)) == 0 for i in range(len(sympl_real_constr))]
constraints += [cp.real(cp.trace(sympl_imag_constr[i]@variable)) == 0 for i in range(len(sympl_imag_constr))]
constraints += [cp.real(cp.trace(def_J_real_constr[i]@variable)) == 0 for i in range(len(def_J_real_constr))]
constraints += [cp.real(cp.trace(def_J_imag_constr[i]@variable)) == 0 for i in range(len(def_J_imag_constr))]
constraints += [cp.real(cp.trace(sdr_def_constr[i]@variable)) == sdr_cst[i] for i in range(len(sdr_def_constr))]
#constraints += [cp.real(cp.trace(inf_constr_photon_nbr[i]@variable)) <= 0 for i in range(len(inf_constr_photon_nbr))]
constraints += [cp.real(cp.trace(real_fixed_pump_constr[i]@variable)) == 0 for i in range(len(real_fixed_pump_constr))]
constraints += [cp.real(cp.trace(imag_fixed_pump_constr[i]@variable)) == 0 for i in range(len(imag_fixed_pump_constr))]
constraints.append(cp.real(cp.trace(obj_f@variable)) >= 0.)
constraints.append(cp.real(cp.trace(photon_nbr_constr@variable)) == 0)
problem = cp.Problem(cp.Minimize(cp.real(cp.trace(obj_f@variable))), constraints=constraints)

In [10]:
problem.solve(solver = "SCS", verbose = True, eps_rel = 10**-8, eps_abs = 10**-8)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Feb 20 03:28:34 PM: Your problem has 1476225 variables, 18128 constraints, and 0 parameters.
(CVXPY) Feb 20 03:28:34 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 20 03:28:34 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 20 03:28:34 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 20 03:28:34 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 20 03:28:35 PM: Compiling problem (target solver=SC

(CVXPY) Feb 20 03:28:40 PM: Applying reduction CvxAttr2Constr
(CVXPY) Feb 20 03:28:40 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Feb 20 04:05:28 PM: Applying reduction SCS
(CVXPY) Feb 20 04:05:31 PM: Finished problem compilation (took 2.217e+03 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Feb 20 04:05:31 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 2952450, constraints m: 2971792
cones: 	  z: primal zero / dual free vars: 18126
	  l: linear vars: 1
	  s: psd vars: 2953665, ssize: 1
settings: eps_abs: 1.0e-08, eps_rel: 1.

-8.894972047812644e-08

In [12]:
# For z_f = 0.25, N_z = 120, le nombre de contraintes non respecter est de 81, but with N_z = 100, it is 48
len([(constraints[i].value(), i) for i in range(len(constraints)) if constraints[i].value() == False])

81

In [54]:
# For z_f = 0.4, N_z = 60, toutes les contraintes sont respecter
len([(constraints[i].value(), i) for i in range(len(constraints)) if constraints[i].value() == False])

48

In [18]:
# Pour z_f = 0.2, N_z = 30, toutes les contraintes sont respecter
[(constraints[i].value(), i) for i in range(len(constraints)) if constraints[i].value() == False]

[]

In [13]:
# Added the fixed pump constraints and seems to have some constraints violated... 
end_product = np.array(variable.value)
[(np.trace(constr_list[i]@end_product), i) for  i in range(len(constr_list)) if np.abs(np.trace(constr_list[i]@end_product)) > 10**-6]

In [16]:
# Get the blocks of the SDP matrix 
test_W_plus = [end_product[(i + 1)*N_omega:(i + 2)*N_omega, (2*N_z + 2)*N_omega:(2*N_z + 3)*N_omega] for i in range(N_z)]
test_W_minus = [end_product[(N_z + i + 1)*N_omega:(N_z + i + 2)*N_omega, (2*N_z + 2)*N_omega:(2*N_z + 3)*N_omega] for i in range(N_z)]
test_W_plus_W_plus_dagger = [end_product[(1 + i)*N_omega:(2 + i)*N_omega,(1 + i)*N_omega:(2 + i)*N_omega] for i in range(N_z)]
test_W_minus_W_minus_dagger = [end_product[(N_z + 1 + i)*N_omega:(N_z + 2 + i)*N_omega,(N_z + 1 + i)*N_omega:(N_z + 2 + i)*N_omega] for i in range(N_z)]
test_J = end_product[:N_omega, (2*N_z + 2)*N_omega:(2*N_z + 3)*N_omega]
test_beta = end_product[(2*N_z + 1)*N_omega:(2*N_z + 2)*N_omega,(2*N_z + 2)*N_omega:(2*N_z + 3)*N_omega]
test_beta_W_plus = [end_product[(2*N_z + 1)*N_omega:(2*N_z + 2)*N_omega,(1 + i)*N_omega:(2 + i)*N_omega] for i in range(N_z)]
test_beta_W_minus = [end_product[(2*N_z + 1)*N_omega:(2*N_z + 2)*N_omega,(N_z + 1 + i)*N_omega:(N_z + 2 + i)*N_omega] for i in range(N_z)]

In [17]:
# the pump is the same as assigned
beta, np.real(test_beta)

(array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318, 0.4236548 ],
        [0.71518937, 0.60276338, 0.54488318, 0.4236548 , 0.64589411],
        [0.60276338, 0.54488318, 0.4236548 , 0.64589411, 0.43758721],
        [0.54488318, 0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
        [0.4236548 , 0.64589411, 0.43758721, 0.891773  , 0.96366276]]),
 array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318, 0.4236548 ],
        [0.71518937, 0.60276338, 0.54488318, 0.4236548 , 0.64589411],
        [0.60276338, 0.54488318, 0.4236548 , 0.64589411, 0.43758721],
        [0.54488318, 0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
        [0.4236548 , 0.64589411, 0.43758721, 0.891773  , 0.96366276]]))

In [41]:
[W_plus[i]@beta for i in range(N_z)]

[array([[0.5488135 +0.j, 0.71518937+0.j, 0.60276338+0.j, 0.54488318+0.j,
         0.4236548 +0.j],
        [0.71518937+0.j, 0.60276338+0.j, 0.54488318+0.j, 0.4236548 +0.j,
         0.64589411+0.j],
        [0.60276338+0.j, 0.54488318+0.j, 0.4236548 +0.j, 0.64589411+0.j,
         0.43758721+0.j],
        [0.54488318+0.j, 0.4236548 +0.j, 0.64589411+0.j, 0.43758721+0.j,
         0.891773  +0.j],
        [0.4236548 +0.j, 0.64589411+0.j, 0.43758721+0.j, 0.891773  +0.j,
         0.96366276+0.j]]),
 array([[0.56007312-7.52833026e-03j, 0.72646659-9.78393902e-03j,
         0.61306789-8.25040666e-03j, 0.5558457 -7.46022353e-03j,
         0.43630457-5.82127382e-03j],
        [0.72651618-4.89713441e-03j, 0.61485587-4.13598597e-03j,
         0.55546245-3.73797651e-03j, 0.43568491-2.90584761e-03j,
         0.65910799-4.40902094e-03j],
        [0.61312365-7.58613459e-06j, 0.55547507-8.09729272e-06j,
         0.43356951-5.14717017e-06j, 0.65621066+4.11675644e-06j,
         0.44984868+1.29372451e-05j],

In [40]:
test_beta_W_plus

[array([[-2.88970896+0.01993903j,  0.96871124+0.04675907j,
          0.78337494+0.05992788j,  0.60946426+0.06901777j,
          0.26740291+0.05813589j],
        [ 0.96306243-0.01413489j, -2.83358082+0.01271947j,
          0.63762048+0.02806741j,  0.31381928+0.03304609j,
          0.70302534+0.06976242j],
        [ 0.77787518-0.03944303j,  0.6368586 -0.01543005j,
         -3.10604007+0.00166475j,  0.84024765+0.02293582j,
          0.31812778+0.02391504j],
        [ 0.60225974-0.05647561j,  0.3124501 -0.02898517j,
          0.84197046-0.02934442j, -3.20807891-0.0090166j ,
          1.2014174 +0.02235964j],
        [ 0.26336087-0.05425434j,  0.70530527-0.07645606j,
          0.32490841-0.04070284j,  1.20592324-0.05375028j,
         -2.2886999 -0.02539853j]]),
 array([[-6.82066799e+00-0.05613323j, -2.21983035e-03+0.08894777j,
         -9.52035169e-03+0.11688003j, -4.24519714e-02+0.13639565j,
         -1.29693739e-01+0.12044426j],
        [-7.11189650e-03-0.03125273j, -6.80918919e+00-0.0239

In [34]:
test_J, J

(array([[ 0.42920144-8.64349819e-13j,  0.08479716+1.36482063e-04j,
          0.09041132+3.65843796e-04j, -0.00731474+1.16364564e-03j,
         -0.01977584+1.67201294e-03j],
        [ 0.08479716-1.36482063e-04j,  0.38955664-5.80279046e-13j,
          0.05852858+3.58464092e-04j,  0.07208778+5.18438441e-04j,
         -0.02198874+1.32172897e-03j],
        [ 0.09041132-3.65843796e-04j,  0.05852858-3.58464092e-04j,
          0.37618934-8.72664647e-14j, -0.01274476+3.83400319e-04j,
          0.03696917+6.32736496e-04j],
        [-0.00731474-1.16364564e-03j,  0.07208778-5.18438441e-04j,
         -0.01274476-3.83400319e-04j,  0.44078336+3.92115234e-13j,
          0.06697447+2.10953028e-04j],
        [-0.01977584-1.67201294e-03j, -0.02198874-1.32172897e-03j,
          0.03696917-6.32736496e-04j,  0.06697447-2.10953028e-04j,
          0.51936651+1.14843970e-12j]]),
 array([[0.37552841+0.j        , 0.38035183-0.07025352j,
         0.33048043-0.1258355j , 0.32240098-0.1998819j ,
         0.31822433

In [23]:
# Test to see if the SDP propagators on linear terms respect the dynamics
# W_plus propagators
test_W_plus = [end_product[(i + 1)*N_omega:(i + 2)*N_omega, (2*N_z + 2)*N_omega:(2*N_z + 3)*N_omega] for i in range(N_z)]
test_W_minus = [end_product[(N_z + i + 1)*N_omega:(N_z + i + 2)*N_omega, (2*N_z + 2)*N_omega:(2*N_z + 3)*N_omega] for i in range(N_z)]
green_fs = api.get_green_f(omega,z)
green_fs.reverse()
green_fs[0] = 0.5*green_fs[0]
green_fs[-1] = 0.5*green_fs[-1]
delta_z = np.abs(z[1] - z[0])
# Not equal
delta_z*sum([green_fs[i]@beta@test_W_plus[i] for i in range(N_z)]) + 2*green_fs[0]
# Last block of W dagger W on primal value
end_product[(N_z - 1)*N_omega:(N_z)*N_omega]

array([[0.3603231 -0.34420143j, 0.07004276-0.02992346j,
        0.06544892-0.01241702j, ..., 0.04486246-0.00441977j,
        0.04218893+0.00862075j, 0.03720372+0.01647132j],
       [0.0644003 -0.03819999j, 0.47022535-0.18843159j,
        0.06264946-0.00652383j, ..., 0.03955272-0.00189544j,
        0.03536173+0.00886392j, 0.04409758+0.02543211j],
       [0.05927041-0.02798392j, 0.06125233-0.01376237j,
        0.48982465-0.00099308j, ..., 0.30324465-0.00107188j,
        0.04221826+0.01266598j, 0.03170701+0.01900081j],
       [0.06137975-0.01754736j, 0.05762755-0.00371591j,
        0.07008552+0.00789881j, ..., 0.04403205+0.00171287j,
        0.28495553+0.11656679j, 0.0483051 +0.0377883j ],
       [0.06015625-0.00689762j, 0.07794502+0.00245061j,
        0.05895218+0.01330016j, ..., 0.03689926+0.00325627j,
        0.05648015+0.02411923j, 0.24456279+0.23591219j]])

In [79]:
test_W_plus[-1]

array([[0.24230649-2.33799883e-01j, 0.04516719-1.45616393e-02j,
        0.0669984 -3.19899826e-03j, 0.05373652+1.10748274e-02j,
        0.03240533+1.47496368e-02j],
       [0.03791743-2.76857544e-02j, 0.33054325-1.37313312e-01j,
        0.05048586-2.64144757e-03j, 0.02879131+5.86965734e-03j,
        0.03232766+1.85312446e-02j],
       [0.05250542-4.00057638e-02j, 0.04718191-1.78949551e-02j,
        0.32531461-2.70243860e-03j, 0.03214193+6.19691159e-03j,
        0.03980129+2.78070971e-02j],
       [0.0458649 -3.00120936e-02j, 0.02778672-1.17931144e-02j,
        0.03304663-3.69168747e-03j, 0.31338471+1.26772057e-01j,
        0.00641843-8.23624869e-05j],
       [0.03183951-1.58119682e-02j, 0.03655777-1.02451898e-02j,
        0.04930651+1.68521636e-03j, 0.00656339-2.09286269e-03j,
        0.23798687+2.37563462e-01j]])

In [1]:
test_W_plus

NameError: name 'test_W_plus' is not defined

In [32]:
J_dagger_J = end_product[:N_omega, :N_omega]
W_plus_dagger_W_plus = []
W_minus_dagger_W_minus = []
for i in range(N_z):
    W_plus_dagger_W_plus.append(end_product[(i + 1)*N_omega:(i + 2)*N_omega, (i + 1)*N_omega:(i + 2)*N_omega])
    W_minus_dagger_W_minus.append(end_product[(i + 1 + N_z)*N_omega:(N_z + i + 2)*N_omega, (N_z + i + 1)*N_omega:(N_z + i + 2)*N_omega])
beta_Z_dagger = end_product[(2*N_z + 2)*N_omega:(2*N_z + 3)*N_omega,(2*N_z + 1)*N_omega:(2*N_z + 2)*N_omega]
np.diag(J_dagger_J), np.diag(J.conj().T@J)

(array([0.00885408+0.j, 0.00885408+0.j, 0.00885408+0.j, 0.00885408+0.j,
        0.00885408+0.j]),
 array([0.0569026 +0.j, 0.03866549+0.j, 0.0317884 +0.j, 0.01701   +0.j,
        0.02705219+0.j]))

In [ ]:
# 10 to the -6??